In [1]:
import bqplot as bqp
import numpy as np
import ipywidgets as widgets
from IPython.display import clear_output, display

## Monte Carlo

**Objective:** the goal of this application is to let you understand the Monte Carlo simulation in a visual way using two simple examples, either rolling a dice or flipping a coin. 

#### How it works
   - **Method**: the method dropdown menu allows you to choose between the two available examples.
   - **MC scatter plot**: for the 'Dice' method, plots the average output from a dice at different number of trials, for example, for four trials and the following outputs (1 x 2, 5 x 1, 6 x 1), the average is $\frac{13}{4}$. For the 'Coin' method, the probability of getting a head is plotted against the number of trials.
   - **MC bar chart**: shows the repititions for each output at the specified number of trials input.
   - **Number of trials**: allows you to set the maximum number of trials to appear on the scatter plot and the results for number of trials to appear on the bar chart. The range of the number of trials is set from 1 to 1000.
   - Refresh the page in order to have a new set of data.
   
As the number of trials increases what can you notice in both figures? What numbers is the MC scatter plot approaching in each method?

In [4]:
# define basic variables
maxtrials = 1000

# variables for Dice method
rolls = np.random.choice([1,2,3,4,5,6], size=maxtrials)
averages = np.cumsum(rolls) / np.arange(1, maxtrials+1)

# variables for Coin method
crolls = np.random.choice(['H', 'T'], size=maxtrials)
Hcount = 0
caverages = np.zeros_like(averages)
for i in range(maxtrials):
    if crolls[i] == 'H':
        Hcount = Hcount + 1
    caverages[i] = Hcount / np.arange(1, maxtrials+1)[i]

# setting the scatter figure objects
sc_x = bqp.LinearScale()
sc_y = bqp.LinearScale()
scat_MC = bqp.Scatter(x=[], y=[], colors=['darkblue'], default_size=1, scales = {'x':sc_x, 'y': sc_y})
ax_x = bqp.Axis(scale=sc_x, label='Number of trials')
ax_y = bqp.Axis(scale=sc_y, orientation='vertical', label='Average of dice output')
fig = bqp.Figure(marks=[scat_MC], axes=[ax_x, ax_y], title='MC scatter plot', animation_duration=1000, preserve_aspect=True)
fig.layout.width = '600px'
fig.layout.length = '500px'

# setting the bar chart figure
sc_xBars = bqp.OrdinalScale()
sc_yBars = bqp.LinearScale()
bars_MC = bqp.Bars(x=['1', '2', '3', '4', '5', '6'], y=[], scales = {'x':sc_xBars, 'y':sc_yBars})
ax_xBars = bqp.Axis(scale=sc_xBars, label='Dice output')
ax_yBars = bqp.Axis(scale=sc_yBars, orientation='vertical', label='Frequency')
figBars = bqp.Figure(marks=[bars_MC], axes=[ax_xBars, ax_yBars], title='MC bar chart', animation_duration=1000, preserve_aspect=True)
figBars.layout.width = '400px'
figBars.layout.length = '300px'

# defining the input widgets
changeset_var = widgets.Dropdown(options=['Dice', 'Coin'], value = 'Dice', description = 'Method')
num_rolls_var = widgets.IntSlider(min=1, max=maxtrials, orientation='vertical', description='Number of trials')

# defining the update function
def on_value_change(*args):
    changeset = changeset_var.value
    num_rolls = num_rolls_var.value
    
    if changeset == 'Dice':
        ax_y.label = 'Average of dice output'
        ax_xBars.label = 'Dice Output'
        scat_MC.y = averages[:num_rolls]
        bkeys, bvalues = np.unique(rolls[:num_rolls], return_counts=True)
    elif changeset == 'Coin':
        ax_y.label = 'Probability of getting a head (H)'
        ax_xBars.label = 'Coin Output'
        scat_MC.y = caverages[:num_rolls]
        bkeys, bvalues = np.unique(crolls[:num_rolls], return_counts=True)
            
    scat_MC.x = np.arange(1, maxtrials+1) [:num_rolls]
    bars_MC.x = bkeys
    bars_MC.y = bvalues
    clear_output(wait=True)

# setting what and how to display
lowerbox = widgets.HBox([fig, figBars, num_rolls_var])
display(widgets.VBox([changeset_var, lowerbox]))

# calling the update function on input change
changeset_var.observe(on_value_change, names = 'value')
num_rolls_var.observe(on_value_change, names = 'value')

# call for the first display
on_value_change(None)